<h2 style='color:purple' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>

In [ ]:
import pandas as pd

In [8]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [9]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [10]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [11]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [12]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [13]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [14]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [15]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [16]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [17]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [19]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [20]:
model.fit(X_train,y_train)

GaussianNB()

In [21]:
model.score(X_test,y_test)

0.7910447761194029

In [22]:
X_test[0:10]

,Pclass,Age,Fare,female
679,1,36.000000,512.3292,0
460,1,48.000000,26.5500,0
230,1,35.000000,83.4750,1
236,2,44.000000,26.0000,0
845,3,42.000000,7.5500,0
271,3,25.000000,0.0000,0
63,3,4.000000,27.9000,0
535,2,7.000000,26.2500,1
768,3,29.699118,24.1500,0
215,1,31.000000,113.2750,1


In [23]:
y_test[0:10]

679    1
460    1
230    1
236    0
845    0
271    1
63     0
535    1
768    0
215    1
Name: Survived, dtype: int64

In [24]:
model.predict(X_test[0:10])

array([1, 0, 1, 0, 0, 0, 0, 1, 0, 1])

In [26]:
model.predict_proba(X_test[:10])

array([[7.38383706e-40, 1.00000000e+00],
       [7.48488955e-01, 2.51511045e-01],
       [1.53580396e-02, 9.84641960e-01],
       [9.18718574e-01, 8.12814256e-02],
       [9.62633248e-01, 3.73667524e-02],
       [9.55787879e-01, 4.42121210e-02],
       [9.02791046e-01, 9.72089540e-02],
       [1.32684826e-01, 8.67315174e-01],
       [9.59176426e-01, 4.08235742e-02],
       [2.30293662e-03, 9.97697063e-01]])

**Calculate the score using cross validation**

In [27]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.696     , 0.752     , 0.824     , 0.77419355, 0.7983871 ])